In [ ]:
#!pip install shutup

In [ ]:
#!pip freeze

In [4]:
#!pip install opencv-python==4.6.0.66
#!pip install matplotlib==3.5.3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#!pip install git+https://github.com/facebookresearch/detectron2.git

In [1]:
#!pip install pillow>=7.1.0

## Cria um .json que precisa para levantar as métricas

In [15]:
# import json
# import os
# from detectron2.structures import BoxMode

# # Defina o caminho para a pasta de anotações e imagens
# annotations_dir = "data/test/labels"
# images_dir = "data/test/images"

# # Lista de imagens no diretório de imagens
# image_file_names = os.listdir(images_dir)

# # Inicialize a lista de anotações
# annotations = []

# # Função para carregar rótulos a partir dos arquivos
# def load_labels(image_name):
#     label_file = os.path.join(test_labels_dir, os.path.splitext(image_name)[0] + ".txt")
#     with open(label_file, "r") as file:
#         labels = file.read().strip().split("\n")
#     labels = [label.split() for label in labels]
#     labels = np.array(labels).astype(float)  # Substitua np.float por float
#     return labels

# # Loop através das imagens
# for image_name in image_file_names:
#     image_path = os.path.join(images_dir, image_name)
#     height, width = cv2.imread(image_path).shape[:2]

#     # Carregue as anotações do arquivo YOLOv5
#     labels = load_labels(image_name)

#     # Converta as anotações do YOLOv5 para o formato BoxMode do Detectron2
#     image_id = image_name  # Use o nome da imagem como ID (você pode ajustar isso conforme necessário)
#     image_info = {
#         "file_name": os.path.join(images_dir, image_name),
#         "height": height,
#         "width": width,
#         "image_id": image_id,
#     }
#     image_info["annotations"] = []

#     for label in labels:
#         x_center, y_center, box_width, box_height, class_id = label
#         x_min = (x_center - box_width / 2) * width
#         y_min = (y_center - box_height / 2) * height
#         x_max = (x_center + box_width / 2) * width
#         y_max = (y_center + box_height / 2) * height

#         annotation = {
#             "bbox": [x_min, y_min, x_max, y_max],
#             "bbox_mode": BoxMode.XYXY_ABS,
#             "category_id": int(class_id),
#         }
#         image_info["annotations"].append(annotation)

#     annotations.append(image_info)

# # Salve as anotações em um arquivo JSON
# output_json_file = "data/test/annotations.json"
# with open(output_json_file, "w") as json_file:
#     json.dump(annotations, json_file)

# print(f"Anotações convertidas e salvas em {output_json_file}")


Anotações convertidas e salvas em data/test/annotations.json


## Gerar .txt com cada um dos modelos Detectron2

In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.data import Metadata

# Define o número do modelo treinado
model_number = "2"  # Altere o número do modelo conforme necessário

# Crie o caminho para o diretório de saída com base no número do modelo
output_dir = f"./output/modelotreinado{model_number}"

# Carregue o modelo treinado
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml'))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.MODEL.WEIGHTS = os.path.join(output_dir, "model_final.pth")
cfg.MODEL.DEVICE = 'cuda'  # Use GPU
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6  # Ajuste o limite NMS conforme necessário
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Ajuste o limite de confiança conforme necessário
cfg.MODEL.ROI_HEADS.NUM_DETECTIONS = 1  # Limite uma detecção por imagem

# Crie um predictor
predictor = DefaultPredictor(cfg)

# Caminho para as imagens de teste
test_images_dir = "./data/test/images"

# Lista de nomes de arquivo de imagens
image_file_names = os.listdir(test_images_dir)

import os

# Criar o diretório de saída, se não existir
output_dir = f"./modelosaida{model_number}"
os.makedirs(output_dir, exist_ok=True)

# Loop sobre as imagens selecionadas
for image_name in image_file_names:
    image_path = os.path.join(test_images_dir, image_name)
    img = cv2.imread(image_path)
    
    # Faça a previsão
    outputs = predictor(img)
        
    # Forçar a seleção da detecção mais confiante
    outputs["instances"] = outputs["instances"][:1]

    # Extrair informações
    pred_boxes = outputs['instances'].to('cpu').pred_boxes.tensor
    pred_classes = outputs['instances'].to('cpu').pred_classes

    # Obter a largura e altura da imagem original
    image_width = img.shape[1]  # Largura da imagem original
    image_height = img.shape[0]  # Altura da imagem original

    # Converter as posições para o formato YOLOv5 (x_center, y_center, width, height)
    box = pred_boxes[0]  # Pegar a primeira (e única) caixa
    x_center = (box[0] + box[2]) / 2 / image_width
    y_center = (box[1] + box[3]) / 2 / image_height
    width = (box[2] - box[0]) / image_width
    height = (box[3] - box[1]) / image_height

    # Resultado a ser salvo no arquivo .txt
    result = f"{pred_classes[0]} {x_center} {y_center} {width} {height}"
    
    # Nome do arquivo .txt a ser salvo
    txt_filename = os.path.splitext(image_name)[0] + ".txt"
    
    # Caminho completo para o arquivo .txt
    txt_filepath = os.path.join(output_dir, txt_filename)
    
    # Salvar o resultado no arquivo .txt
    with open(txt_filepath, "w") as txt_file:
        txt_file.write(result)
    
    # Converter de BGR para RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Definir as classes desejadas
    custom_classes = [
        "BULK CARRIER",
        "CONTAINER SHIP",
        "GENERAL CARGO",
        "OIL PRODUCTS TANKER",
        "PASSENGERS SHIP",
        "TANKER",
        "TRAWLER",
        "TUG",
        "VEHICLES CARRIER",
        "YACHT"
    ]
    
    # Criar um Metadata personalizado
    custom_metadata = Metadata()
    custom_metadata.thing_classes = custom_classes
    
    # Criar o visualizador com o Metadata personalizado
    v = Visualizer(img_rgb, custom_metadata, scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Plote a imagem com as previsões usando matplotlib
    plt.imshow(out.get_image())
    plt.axis('off')
    plt.show()